# AML - VOC 2012 - test set evaluation

## Imports and utility

In [17]:
from google.colab import drive

import numpy as np
import os
import shutil
import tarfile

import tensorflow as tf

In [2]:
IMG_SIZE = (128, 128)

In [3]:
ROOT_DIR = '/content/drive/MyDrive/'
BASE_DIR = ROOT_DIR + 'project/test/'
PARTITION_FILENAMES = [f"voc_test_partition_{i}.npz" for i in range(3)]
PARTITION_PATHS = [os.path.join(BASE_DIR, filename) for filename in PARTITION_FILENAMES]

PARTITION_PATHS

['/content/drive/MyDrive/project/test/voc_test_partition_0.npz',
 '/content/drive/MyDrive/project/test/voc_test_partition_1.npz',
 '/content/drive/MyDrive/project/test/voc_test_partition_2.npz']

In [4]:
def preprocess_images(images: np.ndarray, resize=IMG_SIZE):
    images /= 255.0
    resized_images = []

    for i in range(images.shape[0]):
        resized_image = tf.image.resize(images[i], resize).numpy()
        resized_images.append(resized_image)

    resized_images = np.array(resized_images)
    del images

    return resized_images

In [5]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [22]:
CLASSES = [
    "aeroplane", "bicycle", "bird", "boat", "bottle", "bus", "car", "cat",
    "chair", "cow", "diningtable", "dog", "horse", "motorbike", "person",
    "pottedplant", "sheep", "sofa", "train", "tvmonitor"
]
PRED_FOLDER = "./results/VOC2012/Main"
FILE_FORMAT = "comp1_cls_test_{}.txt"

In [23]:
def serialize_predictions(model_filepath, partition_paths, classes=CLASSES, pred_folder=PRED_FOLDER, file_format=FILE_FORMAT):
  preds = []
  filenames = []

  print(f"Loading model from '{model_filepath}'")
  model = tf.keras.models.load_model(model_filepath)

  for i, partition in enumerate(partition_paths):
    print(f"Partition {i + 1}/{len(partition_paths)}:")
    print(f"\tLoading partition...")
    partition_data = np.load(partition, allow_pickle=True)
    partition_images = partition_data['images']
    partition_filenames = partition_data['filenames']

    print(f"\tPreprocessing partition...")
    partition_images = preprocess_images(partition_images)

    print(f"\tComputing predictions...")
    partition_preds = model.predict(partition_images)
    del partition_images

    preds.append(partition_preds)
    filenames.append(partition_filenames)

  print(f"Concatenating partitions predictions...")
  preds = np.concatenate(preds)
  filenames = np.concatenate(filenames)

  print(f"Sorting predictions by filename...")
  indices = np.argsort(filenames)
  preds = preds[indices]
  filenames = filenames[indices]

  print("Serializing predictions...")
  os.makedirs(pred_folder, exist_ok=True)
  preds_t = preds.T
  for i, c in enumerate(classes):
    file_path = os.path.join(pred_folder, file_format.format(c))
    print(f"\tSerializing predictions for class '{c}' to '{file_path}'")
    f = open(file_path, 'w')
    for filename, pred in zip(filenames, preds_t[i]):
      f.write(f"{filename} {pred}\n")
    f.close()

  del preds, filenames

In [24]:
model_filename = "baseline_augmented_model"

In [25]:
serialize_predictions(f"{model_filename}.keras", PARTITION_PATHS)

Loading model from 'baseline_augmented_model.keras'
Partition 1/3:
	Loading partition...
	Preprocessing partition...
	Computing predictions...
169/169 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step
Partition 2/3:
	Loading partition...
	Preprocessing partition...
	Computing predictions...
169/169 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step
Partition 3/3:
	Loading partition...
	Preprocessing partition...
	Computing predictions...
169/169 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step
Concatenating partitions predictions...
Sorting predictions by filename...
Serializing predictions...
	Serializing predictions for class 'aeroplane' to './results/VOC2012/Main/comp1_cls_test_aeroplane.txt'
	Serializing predictions for class 'bicycle' to './results/VOC2012/Main/comp1_cls_test_bicycle.txt'
	Serializing predictions for class 'bird' to './results/VOC2012/Main/comp1_cls_test_bird.txt'
	Serializing predictions for class 'boat' to './results/VOC2012/Main/comp1_cls_test_boat.txt'
	Serializing predictions for class 'bottle' to './result

In [29]:
output_filename = f'results.tgz'
with tarfile.open(output_filename, "w:gz") as tar:
    tar.add(PRED_FOLDER, arcname=PRED_FOLDER)

destination = BASE_DIR + f"{model_filename}_{output_filename}"
shutil.move(output_filename, destination)

'/content/drive/MyDrive/project/test/baseline_augmented_model_results.tgz'